In [2]:
import pyarrow # must occur prior to ray import
import ray
from ray import tune
from ray.tune import ExperimentAnalysis
from ray.tune.search.hyperopt import HyperOptSearch
import datetime
import numpy as np
import pandas as pd
import random
import seaborn as sns; sns.set()
from collections import Counter
from datasets import load_from_disk
from scipy.stats import ranksums
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertForSequenceClassification
from transformers import Trainer
from transformers.training_args import TrainingArguments

from geneformer import DataCollatorForCellClassification
from geneformer import TranscriptomeTokenizer
import anndata as ad
import pandas as pd

In [3]:
train_dataset=load_from_disk("/root/autodl-tmp/EC.dataset")

In [4]:
num_proc=30
train_dataset

Dataset({
    features: ['input_ids', 'cell_type', 'condition', 'length'],
    num_rows: 3142
})

In [5]:
conditions = train_dataset["condition"]
target_names = list(set(conditions))

In [6]:
target_names

['AF', 'Ctrl']

In [7]:
target_name_id_dict = dict(zip(target_names,[i for i in range(len(target_names))]))

In [8]:
target_name_id_dict

{'AF': 0, 'Ctrl': 1}

In [9]:
trainset_v3 = train_dataset.rename_column("condition","label")

In [10]:
trainset_v3

Dataset({
    features: ['input_ids', 'cell_type', 'label', 'length'],
    num_rows: 3142
})

In [11]:
def classes_to_ids(example):
    example["label"] = target_name_id_dict[example["label"]]
    return example

trainset_v4 = trainset_v3.map(classes_to_ids, num_proc=num_proc)

Map (num_proc=30):   0%|          | 0/3142 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [12]:
indiv_list = trainset_v4["input_ids"]

In [13]:
len(indiv_list)

3142

In [14]:
random.seed(42)
train_indiv = random.sample(indiv_list,round(0.7*len(indiv_list)))
eval_indiv = [indiv for indiv in indiv_list if indiv not in train_indiv]
valid_indiv = random.sample(eval_indiv,round(0.5*len(eval_indiv)))
test_indiv = [indiv for indiv in eval_indiv if indiv not in valid_indiv]

In [15]:
def if_train(example):
    return example["input_ids"] in train_indiv

classifier_trainset = trainset_v4.filter(if_train,num_proc=num_proc).shuffle(seed=42)

Filter (num_proc=30):   0%|          | 0/3142 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root

In [16]:
def if_valid(example):
    return example["input_ids"] in valid_indiv

classifier_validset = trainset_v4.filter(if_valid,num_proc=num_proc).shuffle(seed=42)

Filter (num_proc=30):   0%|          | 0/3142 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root

In [17]:
def if_test(example):
    return example["input_ids"] in test_indiv

classifier_testdset = trainset_v4.filter(if_test,num_proc=num_proc).shuffle(seed=42)

Filter (num_proc=30):   0%|          | 0/3142 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root

In [18]:
current_date = datetime.datetime.now()
datestamp = f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}"
output_dir = f"/root/autodl-tmp/EC/model/{datestamp}_geneformer_DiseaseClassifier/"

In [19]:
import os
saved_model_test = os.path.join(output_dir, f"pytorch_model.bin")
if os.path.isfile(saved_model_test) == True:
    raise Exception("Model already saved to this directory.")

In [20]:
# make output directory
import subprocess
subprocess.call(f'mkdir {output_dir}', shell=True)

0

In [21]:
# set training parameters
# how many pretrained layers to freeze
freeze_layers = 2
# batch size for training and eval
geneformer_batch_size = 12
# number of epochs
epochs = 10
# logging steps
logging_steps = round(len(classifier_trainset)/geneformer_batch_size/10)

In [22]:
def model_init():
    model = BertForSequenceClassification.from_pretrained("/root/autodl-tmp/Geneformer/",
                                                          num_labels=len(target_names),
                                                          output_attentions = False,
                                                          output_hidden_states = False)
    if freeze_layers is not None:
        modules_to_freeze = model.bert.encoder.layer[:freeze_layers]
        for module in modules_to_freeze:
            for param in module.parameters():
                param.requires_grad = False

    return model

In [23]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy and macro f1 using sklearn's function
    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    return {
      'accuracy': acc,
      'macro_f1': macro_f1
    }

In [26]:
training_args = {
    "do_train": True,
    "do_eval": True,
    "evaluation_strategy": "steps",
    "eval_steps": logging_steps,
    "logging_steps": logging_steps,
    "group_by_length": True,
    "length_column_name": "length",
    "disable_tqdm": True,
    "skip_memory_metrics": True, # memory tracker causes errors in raytune
    "per_device_train_batch_size": geneformer_batch_size,
    "per_device_eval_batch_size": geneformer_batch_size,
    "num_train_epochs": epochs,
    "load_best_model_at_end": True,
    "output_dir": output_dir,
    "save_steps":18,
}

In [27]:
training_args_init = TrainingArguments(**training_args)

In [28]:
trainer = Trainer(
    model_init=model_init,
    args=training_args_init,
    data_collator=DataCollatorForCellClassification(),
    train_dataset=classifier_trainset,
    eval_dataset=classifier_validset,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/Geneformer/ and are newly initialized: ['classifier.weight', 'classifier.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
ray_config = {
    "num_train_epochs": tune.choice([epochs]),
    "learning_rate": tune.loguniform(1e-6, 1e-3),
    "weight_decay": tune.uniform(0.0, 0.3),
    "lr_scheduler_type": tune.choice(["linear","cosine","polynomial"]),
    "warmup_steps": tune.uniform(100, 2000),
    "seed": tune.uniform(0,100),
    "per_device_train_batch_size": tune.choice([geneformer_batch_size])
}

In [30]:
hyperopt_search = HyperOptSearch(
    metric="eval_accuracy", mode="max")

In [31]:
# optimize hyperparameters
trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    resources_per_trial={"cpu":8,"gpu":1},
    hp_space=lambda _: ray_config,
    search_alg=hyperopt_search,
    n_trials=100, # number of trials
    progress_reporter=tune.CLIReporter(max_report_frequency=600,
                                                   sort_by_metric=True,
                                                   max_progress_rows=100,
                                                   mode="max",
                                                   metric="eval_accuracy",
                                                   metric_columns=["loss", "eval_loss", "eval_accuracy","macro_f1"])
)

DeprecationWarning: Accepting a `checkpoint_dir` argument in your training function is deprecated.
Please use `ray.train.get_checkpoint()` to access your checkpoint as a
`ray.train.Checkpoint` object instead. See below for an example:

Before
------

from ray import tune

def train_fn(config, checkpoint_dir=None):
    if checkpoint_dir:
        torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))
    ...

tuner = tune.Tuner(train_fn)
tuner.fit()

After
-----

from ray import train, tune

def train_fn(config):
    checkpoint: train.Checkpoint = train.get_checkpoint()
    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))
    ...

tuner = tune.Tuner(train_fn)
tuner.fit()

In [32]:
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/Geneformer/ and are newly initialized: ['classifier.weight', 'classifier.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.69, 'learning_rate': 4.951086956521739e-05, 'epoch': 0.1}
{'eval_loss': 0.7026143670082092, 'eval_accuracy': 0.4470338983050847, 'eval_macro_f1': 0.4470314162211659, 'eval_runtime': 3.0859, 'eval_samples_per_second': 152.956, 'eval_steps_per_second': 12.962, 'epoch': 0.1}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.6899, 'learning_rate': 4.9021739130434785e-05, 'epoch': 0.2}
{'eval_loss': 0.66287761926651, 'eval_accuracy': 0.6313559322033898, 'eval_macro_f1': 0.40780479363156524, 'eval_runtime': 3.0693, 'eval_samples_per_second': 153.779, 'eval_steps_per_second': 13.032, 'epoch': 0.2}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


...

/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.1656, 'learning_rate': 1.0869565217391305e-07, 'epoch': 9.98}
{'eval_loss': 0.4739932417869568, 'eval_accuracy': 0.8008474576271186, 'eval_macro_f1': 0.78896499238965, 'eval_runtime': 3.0753, 'eval_samples_per_second': 153.483, 'eval_steps_per_second': 13.007, 'epoch': 9.98}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'train_runtime': 544.8845, 'train_samples_per_second': 40.357, 'train_steps_per_second': 3.377, 'train_loss': 0.3989227387811179, 'epoch': 10.0}


TrainOutput(global_step=1840, training_loss=0.3989227387811179, metrics={'train_runtime': 544.8845, 'train_samples_per_second': 40.357, 'train_steps_per_second': 3.377, 'train_loss': 0.3989227387811179, 'epoch': 10.0})

In [33]:
predictions = trainer.predict(classifier_testdset)

In [34]:
import pickle
with open(f"{output_dir}predictions.pickle", "wb") as fp:
        pickle.dump(predictions, fp)
trainer.save_metrics("eval",predictions.metrics)
trainer.save_model(output_dir)

In [3]:
from geneformer import InSilicoPerturber
from geneformer import InSilicoPerturberStats

In [4]:
isp = InSilicoPerturber(perturb_type="delete",
                        perturb_rank_shift=None,
                        genes_to_perturb="all",
                        combos=0,
                        anchor_gene=None,
                        model_type="CellClassifier",
                        num_classes=2,
                        emb_mode="cell",
                        cell_emb_style="mean_pool",
                        filter_data=None,  #{"cell_type":["CD8-positive, alpha-beta T cell"]}
                        cell_states_to_model={'state_key': 'condition',
                                              'start_state': 'AF', 
                                              'goal_state': 'Ctrl', 
                                              'alt_states': []},
                        max_ncells=1000,
                        emb_layer=0,
                        forward_batch_size=50,
                        nproc=4)

isp.perturb_data("/root/autodl-tmp/EC/model/231116_geneformer_DiseaseClassifier",
                 "/root/autodl-tmp/EC.dataset",
                 "/root/autodl-tmp/EC/",
                 "EC_perturbation")

Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/3142 [00:00<?, ? examples/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

...

Map (num_proc=4):   0%|          | 0/436 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


Map (num_proc=4):   0%|          | 0/431 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/431 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)
/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1387: FutureWarning: promote has been superseded by mode='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


In [5]:
ispstats = InSilicoPerturberStats(mode="goal_state_shift",
                                  genes_perturbed="all",
                                  combos=0,
                                  anchor_gene=None,
                                  cell_states_to_model={'state_key': 'condition',
                                              'start_state': 'AF', 
                                              'goal_state': 'Ctrl', 
                                              'alt_states': []})

In [6]:
ispstats.get_stats("/root/autodl-tmp/EC",
                   None,
                   "/root/autodl-tmp/EC/",
                   "EC")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/13356 [00:00<?, ?it/s]

  0%|          | 0/13356 [00:00<?, ?it/s]